In [1]:
# Import machine-learning packages
import torch
from torch import nn

# Import plotting packages
from IPython.display import Image, HTML
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import time
import base64
import numpy as np

%matplotlib inline

In [2]:
import xarray as xr

In [3]:
data = xr.open_dataset("../data4training/icepack.h.20150101.nc")

<xarray.Dataset> Size: 49MB
Dimensions:   (ni: 4, ncat: 5, ntrcr: 21, time: 8760)
Coordinates:
  * ni        (ni) int32 16B 1 2 3 4
  * ncat      (ncat) int32 20B 1 2 3 4 5
  * ntrcr     (ntrcr) int32 84B 1 2 3 4 5 6 7 8 9 ... 13 14 15 16 17 18 19 20 21
  * time      (time) object 70kB 2015-01-01 01:00:00 ... 2016-01-01 00:00:00
Data variables: (12/39)
    timestep  (time) int32 35kB ...
    date      (time) float64 70kB ...
    aice      (time, ni) float64 280kB ...
    vice      (time, ni) float64 280kB ...
    vsno      (time, ni) float64 280kB ...
    uvel      (time, ni) float64 280kB ...
    ...        ...
    fhocn     (time, ni) float64 280kB ...
    aicen     (time, ncat, ni) float64 1MB ...
    vicen     (time, ncat, ni) float64 1MB ...
    vsnon     (time, ncat, ni) float64 1MB ...
    trcr      (time, ntrcr, ni) float64 6MB ...
    trcrn     (time, ncat, ntrcr, ni) float64 29MB ...

In [ ]:
n = 24*365 - 24*60
x = torch.ones(n,10)
y = torch.ones(n,1)